<a href="https://colab.research.google.com/github/seunghee0518/R_File/blob/main/R_%EB%8B%A4%EB%B3%80%EB%9F%89%EB%B6%84%EC%84%9D_7%EC%9E%A5_%ED%8C%90%EB%B3%84%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사후확률을 이용한 판별분석
* 판별점수 D = d로 주어졌을 때, 그룹 g 속할 확률 P(G = g | D = d)을 의미하는 것으로 이 값이 큰 그룹에 배정하는 방법

In [1]:
data7_url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%EB%8B%A4%EB%B3%80%EB%9E%91%EB%B6%84%EC%84%9D/mvadata/data7-1.txt'
data7 = read.table(data7_url, header=T)

In [2]:
head(data7)

,group,x1,x2,x3
,<chr>,<int>,<int>,<int>
1,g1,98,81,38
2,g1,103,84,38
3,g1,103,86,42
4,g1,105,86,42
5,g1,109,88,44
6,g1,123,92,50


In [3]:
#패키지 가져오기
library(MASS)

In [4]:
data7.lda = lda(group ~ ., data = data7)
pred.lda = predict(data7.lda, newdata = data7)
pred.lda$class

[1] g2 g2 g1 g1 g1 g1 g1 g2 g2 g2 g1 g2 g2
Levels: g1 g2

In [5]:
pred.lda$posterior

,g1,g2
1,0.35014011,0.64985989
2,0.15881893,0.84118107
3,0.97194803,0.02805197
4,0.92101209,0.07898791
5,0.96899785,0.03100215
6,0.98515585,0.01484415
7,0.97539841,0.02460159
8,0.05593911,0.94406089
9,0.03967580,0.96032420
10,0.04071603,0.95928397


#판별모형 평가(람다 통계량 이용)

In [6]:
confm.lda = table(data7$group, pred.lda$class)
confm.lda

    
     g1 g2
  g1  5  2
  g2  1  5

# 판별분석 예제


In [7]:
#알콜자료
alcohol_url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%EB%8B%A4%EB%B3%80%EB%9E%91%EB%B6%84%EC%84%9D/mvadata/alcohol.csv'
alcohol = read.csv(alcohol_url, header = T)
head(alcohol)

,TYPE,MEOH,ACET,BU1,MEPR,ACAL,LNPRO1
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,typeA,3,15,0.2,9,9.0,5.86
2,typeA,23,13,0.8,9,2.0,6.67
3,typeA,65,96,0.4,9,4.0,5.31
4,typeA,279,66,0.9,36,4.8,5.45
5,typeA,292,210,1.1,34,8.0,4.08
6,typeA,371,414,1.2,39,9.0,6.22


In [8]:
summary(alcohol)

     TYPE                MEOH             ACET            BU1       
 Length:77          Min.   :   3.0   Min.   : 13.0   Min.   : 0.20  
 Class :character   1st Qu.: 620.0   1st Qu.:127.0   1st Qu.: 9.30  
 Mode  :character   Median : 910.0   Median :181.0   Median :17.00  
                    Mean   : 845.6   Mean   :211.7   Mean   :14.74  
                    3rd Qu.:1087.0   3rd Qu.:287.0   3rd Qu.:20.00  
                    Max.   :1548.0   Max.   :495.0   Max.   :30.00  
      MEPR            ACAL           LNPRO1     
 Min.   : 9.00   Min.   : 2.00   Min.   :3.300  
 1st Qu.:26.00   1st Qu.: 8.60   1st Qu.:4.280  
 Median :33.00   Median :11.00   Median :5.260  
 Mean   :35.76   Mean   :12.51   Mean   :5.301  
 3rd Qu.:46.00   3rd Qu.:15.00   3rd Qu.:6.220  
 Max.   :72.00   Max.   :28.00   Max.   :8.010  

In [9]:
#패키지 가져오기
library(MASS)

In [10]:
#판별분석 실행
alcohol.lda = lda(TYPE ~ ., data = alcohol)
alcohol.lda

Call:
lda(TYPE ~ ., data = alcohol)

Prior probabilities of groups:
    typeA     typeB     typeC 
0.2337662 0.3766234 0.3896104 

Group means:
           MEOH     ACET       BU1     MEPR     ACAL   LNPRO1
typeA  378.6944 218.0167  1.511111 32.06667 11.16667 6.231111
typeB  939.1379 247.3448 17.906897 30.55172 12.54138 4.883103
typeC 1035.4000 173.3667 19.620000 43.00000 13.27333 5.145667

Coefficients of linear discriminants:
                 LD1           LD2
MEOH    3.382089e-03  0.0005710473
ACET   -4.649248e-05 -0.0066573606
BU1     1.322048e-01 -0.0162598664
MEPR   -2.562255e-02  0.0533609640
ACAL   -4.048757e-02  0.0297883525
LNPRO1 -2.791911e-01  0.3894400487

Proportion of trace:
   LD1    LD2 
0.9168 0.0832 

* Prior prob ... (각 그룹별 개수에 비례한 사전확률값)
* 그룹수가 3개 이므로 2개의 선형판별함수가 출력됨

In [11]:
#분류하기
alcohol.pred.lda = predict(alcohol.lda, newdata = alcohol)
names(alcohol.pred.lda)

[1] "class"     "posterior" "x"

In [12]:
head(alcohol.pred.lda$class)

[1] typeA typeA typeA typeA typeA typeA
Levels: typeA typeB typeC

In [29]:
head(alcohol.pred.lda$posterior)

,typeA,typeB,typeC
1,0.9999932,4.878233e-06,1.882559e-06
2,0.9999838,1.104519e-05,5.137599e-06
3,0.9998621,1.216151e-04,1.626668e-05
4,0.9999067,3.768926e-05,5.562616e-05
5,0.9993267,5.527433e-04,1.205654e-04
6,0.9999181,7.058307e-05,1.131889e-05


In [13]:
head(alcohol.pred.lda$x)

,LD1,LD2
1,-4.091901,-0.2497072
2,-3.887575,-0.1277995
3,-3.503543,-1.1199342
4,-3.475565,0.7129591
5,-3.107675,-0.7864612
6,-3.602823,-0.9710812


In [14]:
#분류표
alcohol.confm.lda = table(alcohol$TYPE, alcohol.pred.lda$class)
alcohol.confm.lda

       
        typeA typeB typeC
  typeA    18     0     0
  typeB     0    23     6
  typeC     0     9    21

In [15]:
#오분류율(error rate)
error = 1 - sum(diag(alcohol.confm.lda))/sum(alcohol.confm.lda)
error

[1] 0.1948052

In [16]:
#판별변수 선택하기
install.packages('klaR')
library(klaR)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘sass’, ‘jquerylib’, ‘R.methodsS3’, ‘R.oo’, ‘R.utils’, ‘proxy’, ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘promises’, ‘bslib’, ‘R.cache’, ‘e1071’, ‘shiny’, ‘miniUI’, ‘styler’, ‘classInt’, ‘labelled’, ‘combinat’, ‘questionr’




In [17]:
alcohol.forward = greedy.wilks(TYPE~., data = alcohol, niveau = 0.01)
alcohol.forward

Formula containing included variables: 

TYPE ~ BU1 + MEPR + MEOH
<environment: 0x563e4f544a18>


Values calculated in each step of the selection procedure: 

  vars Wilks.lambda F.statistics.overall p.value.overall F.statistics.diff
1  BU1    0.2989919             86.74917    3.975561e-20         86.749173
2 MEPR    0.2489312             36.65654    3.402765e-21          7.340239
3 MEOH    0.2012539             29.49819    7.226240e-23          8.528444
  p.value.diff
1 3.975561e-20
2 1.235768e-03
3 4.692630e-04

In [18]:
#변수선택 후 판별분석 실행
alcohol.fwd.lda = lda(alcohol.forward$formula, data = alcohol)
alcohol.fwd.lda

Call:
lda(alcohol.forward$formula, data = alcohol)

Prior probabilities of groups:
    typeA     typeB     typeC 
0.2337662 0.3766234 0.3896104 

Group means:
            BU1     MEPR      MEOH
typeA  1.511111 32.06667  378.6944
typeB 17.906897 30.55172  939.1379
typeC 19.620000 43.00000 1035.4000

Coefficients of linear discriminants:
              LD1           LD2
BU1   0.149874251 -0.0313069489
MEPR -0.025447467  0.0759388778
MEOH  0.002809521  0.0002065334

Proportion of trace:
   LD1    LD2 
0.9516 0.0484 

In [20]:
#변수선택 후 분류표
alcohol.pred.fwd.lda = predict(alcohol.fwd.lda, newdata = alcohol)
confm.fwd = table(alcohol$TYPE, alcohol.pred.fwd.lda$class)
confm.fwd

       
        typeA typeB typeC
  typeA    18     0     0
  typeB     0    22     7
  typeC     1    12    17

In [21]:
error2 = 1 - sum(diag(confm.fwd))/sum(confm.fwd)
error2

[1] 0.2597403

In [26]:
#분류함수 구하기(R은 분류함수를 제공하는 함수가 없으므로 따로 function문 만듦)
classfunc.lda <- function(x, groups) {
  x.lda <- lda(groups ~ ., as.data.frame(x))

  gr <- length(unique(groups)) # groups might be factors or numeric
  v <- ncol(x)                 # variables
  m <- x.lda$means             # group meas

  w <- array(NA, dim = c(v, v, gr))

  tot.len <- nrow(x)
  subgr.len <- array(0, gr)

  for (i in 1:gr){
    tmp <- scale(subset(x, groups == unique(groups)[i]), scale = FALSE)
    w[,,i] <- t(tmp) %*% tmp
    subgr.len[i] <- nrow(tmp)
  }

  W <- w[, , 1]
  for (i in 2:gr)
   W <- W + w[, , i]

  V <- W/(nrow(x) - gr)
  iV <- solve(V)

  class.func <- matrix(NA, nrow = v + 1, ncol = gr)

  colnames(class.func) <- rownames(m)
  rownames(class.func) <- c('constant', colnames(m))

  for(i in 1:gr) {
    ni <- subgr.len[i]
    class.func[1, i] <- -0.5 * t(m[i,]) %*% iV %*% (m[i,]) + log(ni / tot.len)
    class.func[2:(v+1), i] <- iV %*% (m[i,])

  }
  x.lda$class.func <- class.func
  
  return(x.lda)
}

In [27]:
#분류함수 계수
X = alcohol[, -1]
classfunc.result = classfunc.lda(X, alcohol$TYPE)
names(classfunc.result)

[1] "prior"      "counts"     "means"      "scaling"    "lev"       
 [6] "svd"        "N"          "call"       "terms"      "xlevels"   
[11] "class.func"

In [28]:
classfunc.result$class.func

,typeA,typeB,typeC
constant,-2.468616e+01,-25.14175440,-29.631643758
MEOH,6.590818e-04,0.01520849,0.016407192
ACET,4.449847e-04,0.00494391,-0.003769616
BU1,-3.934206e-02,0.55665395,0.553048496
MEPR,1.868923e-01,0.03590143,0.102270552
ACAL,3.917393e-02,-0.16088059,-0.127328438
LNPRO1,6.378935e+00,4.86936954,5.341430073


> 각 그룹에 대한 분류함수 계수
* typeA 분류함수 = -24.68616 + 0.000659 MEOH + 0.000445 ACET - 0.039342 BUI + 0.186892 MEPR + 0.039174 ACAL + 6.378935 LNPROI
* 첫번째 케이스의 경우 (3.0,15.0, 0.2, 9.0, 9.0, 5.86)
* 해당 케이스를 대입할 경우 첫번째 분류함수값
* (typeA, typeB, typeC) = (14.72977, 2.499048, 1.546903)
* 따라서 첫번째 케이스는 가장 큰값을 갖는 typeA로 분류됨

In [30]:
head(alcohol.pred.lda$posterior)

,typeA,typeB,typeC
1,0.9999932,4.878233e-06,1.882559e-06
2,0.9999838,1.104519e-05,5.137599e-06
3,0.9998621,1.216151e-04,1.626668e-05
4,0.9999067,3.768926e-05,5.562616e-05
5,0.9993267,5.527433e-04,1.205654e-04
6,0.9999181,7.058307e-05,1.131889e-05


* 똑같이 typeA로 구분됨